# Dataframes and scikit-learn

In [1]:
# this cell's metadata contains
# "nbsphinx": "hidden" so it is hidden by nbsphinx
PATH_YIELD_ENGINE = 'src'
def set_paths() -> None:
    import sys
    import os
    
    if 'cwd' not in globals():
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir)
        os.chdir(cwd)   
    print(f"working dir is '{os.getcwd()}'")                            
    if PATH_YIELD_ENGINE not in sys.path:
        sys.path.insert(0, PATH_YIELD_ENGINE)
    print(f"added `{sys.path[0]}` to python paths")
set_paths()

working dir is 'C:\Users\martin florent\Documents\projects\yield-engine'
added `src` to python paths


In [68]:
import pandas as pd
# todo remove RandomForestRegressor when possible
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from gamma import Sample
from gamma.viz.dendrogram import DendrogramDrawer, FeatMapStyle, LineStyle
from gamma.sklearndf.pipeline import PipelineDF
from gamma.sklearndf.transformation import SimpleImputerDF, MissingIndicatorDF, OneHotEncoderDF, ColumnTransformerDF
from gamma.sklearndf.transformation.extra import BorutaDF
from gamma.sklearndf.regression import RandomForestRegressorDF, LGBMRegressorDF
from gamma.model.inspection import ModelInspector
from gamma.model.prediction import PredictorFitCV
from gamma.model.selection import ModelPipelineDF, ModelGrid, ModelRanker, summary_report
from gamma.model.validation import CircularCrossValidator
from gamma.yieldengine.simulation import UnivariateSimulator
from gamma.yieldengine.partition import ContinuousRangePartitioning
from gamma.viz.simulation import SimulationDrawer, SimulationMatplotStyle

In [41]:
df = pd.read_csv('data/ames-housing-dataset/train.csv')
df = df.drop(['Id', 'YrSold', 'MoSold'], axis=1)
TARGET = "SalePrice"

## Example with One Hot Encoding

TODO

Some features are categorical, like **GarageType**:

In [42]:
df.GarageType.unique()

array(['Attchd', 'Detchd', 'BuiltIn', 'CarPort', nan, 'Basment', '2Types'],
      dtype=object)

On top of that there are missing values:

In [58]:
df.GarageType.isna().sum()

81

With scikit-learn to one-hot encode one should first impute missing values, and then one hot encode categorical variables:

In [59]:
categorical_features = df.select_dtypes([object]).columns

In [63]:
ohe_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='nan')),
    ('ohe', OneHotEncoder(sparse=False))
])

In [64]:
ohe_transformer = ColumnTransformer(
    transformers=[('ohe', ohe_pipeline, categorical_features)],
    remainder='passthrough'
)

In [65]:
ohe_transformer.fit_transform(df)

array([[     0.,      0.,      0., ...,      0.,      0., 208500.],
       [     0.,      0.,      0., ...,      0.,      0., 181500.],
       [     0.,      0.,      0., ...,      0.,      0., 223500.],
       ...,
       [     0.,      0.,      0., ...,      0.,   2500., 266500.],
       [     0.,      0.,      0., ...,      0.,      0., 142125.],
       [     0.,      0.,      0., ...,      0.,      0., 147500.]])

In [70]:
ohe_pipeline_df = PipelineDF(steps=[
    ('imputer', SimpleImputerDF(strategy='constant', fill_value='nan')),
    ('ohe', OneHotEncoderDF(sparse=False))
])

In [71]:
ohe_transformer_df = ColumnTransformerDF(
    transformers=[('ohe_df', ohe_pipeline_df, categorical_features)],
    remainder='passthrough'
)

ValueError: arg column_transformer with unsupported remainder attribute (passthrough)

In [65]:
ohe_transformer.fit_transform(df)

array([[     0.,      0.,      0., ...,      0.,      0., 208500.],
       [     0.,      0.,      0., ...,      0.,      0., 181500.],
       [     0.,      0.,      0., ...,      0.,      0., 223500.],
       ...,
       [     0.,      0.,      0., ...,      0.,   2500., 266500.],
       [     0.,      0.,      0., ...,      0.,      0., 142125.],
       [     0.,      0.,      0., ...,      0.,      0., 147500.]])

In [30]:
df.GarageType.isna().sum()

81

In [31]:
df = SimpleImputerDF(strategy='constant', fill_value='nan').fit_transform(df)

In [ ]:
df.GarageType

In [25]:
ohe = OneHotEncoder(categories=['GarageType'])

In [26]:
ohe.fit_transform(df)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [15]:
df.RoofStyle.value_counts()

Gable      1141
Hip         286
Flat         13
Gambrel      11
Mansard       7
Shed          2
Name: RoofStyle, dtype: int64

## Transformer

TODO

In [ ]:
num_features = df.select_dtypes('number').columns
df = df[list(num_features)]

## Regressor

TODO

## Classifier

TODO

## Pipeline

TODO

## Extra

TODO
Boruta, LGBM...